# Pointer

In [1]:
import json
import sys

from konfoo import *

## YAML Support

In [2]:
import oyaml as yaml

In [3]:
def to_yaml(container, *attributes, **options):
    flow_style = options.pop('flow_style', False)
    return yaml.safe_dump(container.view_fields(*attributes, **options), 
                          stream=sys.stdout, 
                          default_flow_style=flow_style)

## Define a Data Object

In [4]:
class DataObject(Structure):
    def __init__(self):
        super().__init__()
        self.size = Decimal(16)
        self.item = Pointer()
        self.index_fields()

In [5]:
data_object = DataObject()

In [6]:
data_object.to_list()

[('DataObject.size', 0), ('DataObject.item', '0x0')]

In [7]:
data_object.to_csv()

[{'id': 'DataObject.size', 'value': 0},
 {'id': 'DataObject.item', 'value': '0x0'}]

In [8]:
data_object.to_json()

'{"size": 0, "item": "0x0"}'

In [9]:
to_yaml(data_object, flow_style=False)

size: 0
item: '0x0'


In [10]:
data_object.to_list(nested=True)

[('DataObject.size', 0), ('DataObject.item', '0x0')]

In [11]:
data_object.to_csv(nested=True)

[{'id': 'DataObject.size', 'value': 0},
 {'id': 'DataObject.item', 'value': '0x0'}]

In [12]:
data_object.to_json(nested=True)

'{"size": 0, "item": {"value": "0x0", "data": null}}'

In [13]:
to_yaml(data_object, nested=True, flow_style=False)

size: 0
item:
  value: '0x0'
  data: null


## Define a Data Object Pointer

In [14]:
class DataObjectPointer(Pointer):

    def __init__(self, address=None, byte_order=BYTEORDER):
        super().__init__(template=DataObject(),
                         address=address,
                         data_order=byte_order)

In [15]:
pointer = DataObjectPointer()

In [16]:
pointer.to_list()

[('DataObjectPointer.field', '0x0'),
 ('DataObjectPointer.data.size', 0),
 ('DataObjectPointer.data.item', '0x0')]

In [17]:
pointer.to_csv()

[{'id': 'DataObjectPointer.field', 'value': '0x0'},
 {'id': 'DataObjectPointer.data.size', 'value': 0},
 {'id': 'DataObjectPointer.data.item', 'value': '0x0'}]

In [18]:
pointer.to_json()

'{"value": "0x0", "data": {"size": 0, "item": "0x0"}}'

In [19]:
to_yaml(pointer, flow_style=False)

value: '0x0'
data:
  size: 0
  item: '0x0'


In [20]:
pointer.to_list(nested=True)

[('DataObjectPointer.field', '0x0'),
 ('DataObjectPointer.data.size', 0),
 ('DataObjectPointer.data.item', '0x0')]

In [21]:
pointer.to_csv(nested=True)

[{'id': 'DataObjectPointer.field', 'value': '0x0'},
 {'id': 'DataObjectPointer.data.size', 'value': 0},
 {'id': 'DataObjectPointer.data.item', 'value': '0x0'}]

In [22]:
pointer.to_json(nested=True)

'{"value": "0x0", "data": {"size": 0, "item": {"value": "0x0", "data": null}}}'

In [23]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x0'
data:
  size: 0
  item:
    value: '0x0'
    data: null


## Nest Pointers

### Nested `pointer` with no `data object`.

In [24]:
pointer = Pointer(Pointer())

In [25]:
pointer.to_list()

[('Pointer.field', '0x0'), ('Pointer.data', '0x0')]

In [26]:
pointer.to_csv()

[{'id': 'Pointer.field', 'value': '0x0'},
 {'id': 'Pointer.data', 'value': '0x0'}]

In [27]:
pointer.to_json()

'{"value": "0x0", "data": "0x0"}'

In [28]:
to_yaml(pointer, flow_style=False)

value: '0x0'
data: '0x0'


In [29]:
pointer.to_list(nested=True)

[('Pointer.field', '0x0'), ('Pointer.data', '0x0')]

In [30]:
pointer.to_csv(nested=True)

[{'id': 'Pointer.field', 'value': '0x0'},
 {'id': 'Pointer.data', 'value': '0x0'}]

In [31]:
pointer.to_json(nested=True)

'{"value": "0x0", "data": {"value": "0x0", "data": null}}'

In [32]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x0'
data:
  value: '0x0'
  data: null


### Nested `pointer` with an `data object`

In [33]:
pointer = Pointer(Pointer(Byte()))

In [34]:
pointer.to_list()

[('Pointer.field', '0x0'), ('Pointer.data', '0x0')]

In [35]:
pointer.to_csv()

[{'id': 'Pointer.field', 'value': '0x0'},
 {'id': 'Pointer.data', 'value': '0x0'}]

In [36]:
pointer.to_json()

'{"value": "0x0", "data": "0x0"}'

In [37]:
to_yaml(pointer, flow_style=False)

value: '0x0'
data: '0x0'


In [38]:
pointer.to_list(nested=True)

[('Pointer.field', '0x0'),
 ('Pointer.data', '0x0'),
 ('Pointer.data.data', '0x0')]

In [39]:
pointer.to_csv(nested=True)

[{'id': 'Pointer.field', 'value': '0x0'},
 {'id': 'Pointer.data', 'value': '0x0'},
 {'id': 'Pointer.data.data', 'value': '0x0'}]

In [40]:
pointer.to_json(nested=True)

'{"value": "0x0", "data": {"value": "0x0", "data": "0x0"}}'

In [41]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x0'
data:
  value: '0x0'
  data: '0x0'


## Access the Data Object

In [42]:
pointer = Pointer(Pointer(Byte()))

In [43]:
pointer.data

Pointer(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=32, value='0x0')

In [44]:
pointer.data.data

Byte(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=1, bit_offset=0), bit_size=8, value='0x0')

## Address of the Data Object

In [45]:
pointer.address

0

## Byte Order of the Data Object

In [46]:
pointer.data_byte_order

Byteorder.little = 'little'

In [47]:
pointer.data_byte_order = 'big'

In [48]:
pointer.data_byte_order

Byteorder.big = 'big'

## Byte Stream for the Data Object

In [49]:
pointer.bytestream

''

In [50]:
pointer.bytestream = '01020304'

In [51]:
pointer.bytestream

'01020304'

## Initialize a Pointer

In [52]:
pointer = Pointer(
    Structure(
        size=Decimal(16),
        item=Pointer(String(14))))

In [53]:
pointer.to_list(nested=True)

[('Pointer.field', '0x0'),
 ('Pointer.data.size', 0),
 ('Pointer.data.item', '0x0'),
 ('Pointer.data.item.data', '')]

In [54]:
pointer.to_csv(nested=True)

[{'id': 'Pointer.field', 'value': '0x0'},
 {'id': 'Pointer.data.size', 'value': 0},
 {'id': 'Pointer.data.item', 'value': '0x0'},
 {'id': 'Pointer.data.item.data', 'value': ''}]

In [55]:
pointer.to_json(nested=True)

'{"value": "0x0", "data": {"size": 0, "item": {"value": "0x0", "data": ""}}}'

In [56]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x0'
data:
  size: 0
  item:
    value: '0x0'
    data: ''


In [57]:
pointer.initialize_fields({
    'value': 0x1,
    'data': {
        'size': 14,
        'item': {'value': 0x10,
                 'data': 'Konfoo is Fun'}}})

In [58]:
pointer.to_list(nested=True)

[('Pointer.field', '0x1'),
 ('Pointer.data.size', 14),
 ('Pointer.data.item', '0x10'),
 ('Pointer.data.item.data', 'Konfoo is Fun')]

In [59]:
pointer.to_csv(nested=True)

[{'id': 'Pointer.field', 'value': '0x1'},
 {'id': 'Pointer.data.size', 'value': 14},
 {'id': 'Pointer.data.item', 'value': '0x10'},
 {'id': 'Pointer.data.item.data', 'value': 'Konfoo is Fun'}]

In [60]:
pointer.to_json(nested=True)

'{"value": "0x1", "data": {"size": 14, "item": {"value": "0x10", "data": "Konfoo is Fun"}}}'

In [61]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x1'
data:
  size: 14
  item:
    value: '0x10'
    data: Konfoo is Fun


## Display a Pointer

In [62]:
pointer = Pointer(
    Structure(
        size=Decimal(16),
        item=Pointer(String())))

In [63]:
pointer.index_fields()

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [64]:
pointer

Pointer(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=32, value='0x0')

## Display the Data Object

In [65]:
pointer.data

{'size': Decimal(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=2, bit_offset=0), bit_size=16, value=0),
 'item': Pointer(index=Index(byte=2, bit=0, address=2, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=32, value='0x0')}

## Metadata of a Pointer

In [66]:
pointer.describe()

{'address': 0,
 'alignment': [4, 0],
 'class': 'Pointer',
 'index': [0, 0],
 'max': 4294967295,
 'min': 0,
 'name': 'Pointer',
 'order': 'auto',
 'signed': False,
 'size': 32,
 'type': 'Pointer',
 'value': '0x0',
 'member': [{'class': 'Structure',
   'name': 'data',
   'size': 2,
   'type': 'Structure',
   'member': [{'address': 0,
     'alignment': [2, 0],
     'class': 'Decimal16',
     'index': [0, 0],
     'max': 65535,
     'min': 0,
     'name': 'size',
     'order': 'auto',
     'signed': False,
     'size': 16,
     'type': 'Field',
     'value': 0},
    {'address': 2,
     'alignment': [4, 0],
     'class': 'Pointer',
     'index': [2, 0],
     'max': 4294967295,
     'min': 0,
     'name': 'item',
     'order': 'auto',
     'signed': False,
     'size': 32,
     'type': 'Pointer',
     'value': '0x0',
     'member': [{'address': 0,
       'alignment': [0, 0],
       'class': 'String',
       'index': [0, 0],
       'name': 'data',
       'order': 'auto',
       'size': 0,
   

In [67]:
json.dump(pointer.describe(), sys.stdout, indent=2)

{
  "address": 0,
  "alignment": [
    4,
    0
  ],
  "class": "Pointer",
  "index": [
    0,
    0
  ],
  "max": 4294967295,
  "min": 0,
  "name": "Pointer",
  "order": "auto",
  "signed": false,
  "size": 32,
  "type": "Pointer",
  "value": "0x0",
  "member": [
    {
      "class": "Structure",
      "name": "data",
      "size": 2,
      "type": "Structure",
      "member": [
        {
          "address": 0,
          "alignment": [
            2,
            0
          ],
          "class": "Decimal16",
          "index": [
            0,
            0
          ],
          "max": 65535,
          "min": 0,
          "name": "size",
          "order": "auto",
          "signed": false,
          "size": 16,
          "type": "Field",
          "value": 0
        },
        {
          "address": 2,
          "alignment": [
            4,
            0
          ],
          "class": "Pointer",
          "index": [
            2,
            0
          ],
          "max": 42949

In [68]:
d3flare_json(pointer.describe(), sys.stdout, indent=2)

{
  "class": "Pointer",
  "name": "Pointer",
  "children": [
    {
      "class": "Pointer",
      "name": "*Pointer",
      "size": 32,
      "value": "0x0"
    },
    {
      "class": "Structure",
      "name": "data",
      "children": [
        {
          "class": "Decimal16",
          "name": "size",
          "size": 16,
          "value": 0
        },
        {
          "class": "Pointer",
          "name": "item",
          "children": [
            {
              "class": "Pointer",
              "name": "*item",
              "size": 32,
              "value": "0x0"
            },
            {
              "class": "String",
              "name": "data",
              "size": 0,
              "value": ""
            }
          ]
        }
      ]
    }
  ]
}

## Size of the Data Object

In [69]:
pointer.data_size

6

## Indexing

In [70]:
pointer = Pointer(
    Structure(
        size=Decimal(16),
        item=Pointer(String(14))))

In [71]:
pointer.initialize_fields({
    'value': 0x1,
    'data': {
        'size': 14,
        'item': {'value': 0x10,
                 'data': 'Konfoo is Fun'}}})

In [72]:
pointer.to_list('index', nested=True)

[('Pointer.field',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.size',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.item',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.item.data',
  Index(byte=0, bit=0, address=0, base_address=0, update=False))]

In [73]:
pointer.to_csv('index.byte', 'index.bit', 'index.address', fieldnames=('id', 'index', 'offset', 'address'), nested=True)

[{'id': 'Pointer.field', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Pointer.data.size', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Pointer.data.item', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Pointer.data.item.data', 'index': 0, 'offset': 0, 'address': 0}]

In [74]:
pointer.to_json('index', nested=True)

'{"value": [0, 0, 0, 0, false], "data": {"size": [0, 0, 0, 0, false], "item": {"value": [0, 0, 0, 0, false], "data": [0, 0, 0, 0, false]}}}'

In [75]:
pointer.index_fields(index=Index())

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [76]:
pointer.index_fields()

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [77]:
pointer.to_list('index', nested=True)

[('Pointer.field',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.size',
  Index(byte=0, bit=0, address=1, base_address=1, update=False)),
 ('Pointer.data.item',
  Index(byte=2, bit=0, address=3, base_address=1, update=False)),
 ('Pointer.data.item.data',
  Index(byte=0, bit=0, address=0, base_address=0, update=False))]

In [78]:
pointer.to_csv('index.byte', 'index.bit', 'index.address', fieldnames=('id', 'index', 'offset', 'address'), nested=True)

[{'id': 'Pointer.field', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Pointer.data.size', 'index': 0, 'offset': 0, 'address': 1},
 {'id': 'Pointer.data.item', 'index': 2, 'offset': 0, 'address': 3},
 {'id': 'Pointer.data.item.data', 'index': 0, 'offset': 0, 'address': 0}]

In [79]:
pointer.to_json('index', nested=True)

'{"value": [0, 0, 0, 0, false], "data": {"size": [0, 0, 1, 1, false], "item": {"value": [2, 0, 3, 1, false], "data": [0, 0, 0, 0, false]}}}'

In [80]:
pointer.index_fields(index=Index(), nested=True)

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [81]:
pointer.index_fields(nested=True)

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [82]:
pointer.to_list('index', nested=True)

[('Pointer.field',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.size',
  Index(byte=0, bit=0, address=1, base_address=1, update=False)),
 ('Pointer.data.item',
  Index(byte=2, bit=0, address=3, base_address=1, update=False)),
 ('Pointer.data.item.data',
  Index(byte=0, bit=0, address=16, base_address=16, update=False))]

In [83]:
pointer.to_csv('index.byte', 'index.bit', 'index.address', fieldnames=('id', 'index', 'offset', 'address'), nested=True)

[{'id': 'Pointer.field', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Pointer.data.size', 'index': 0, 'offset': 0, 'address': 1},
 {'id': 'Pointer.data.item', 'index': 2, 'offset': 0, 'address': 3},
 {'id': 'Pointer.data.item.data', 'index': 0, 'offset': 0, 'address': 16}]

In [84]:
pointer.to_json('index', nested=True)

'{"value": [0, 0, 0, 0, false], "data": {"size": [0, 0, 1, 1, false], "item": {"value": [2, 0, 3, 1, false], "data": [0, 0, 16, 16, false]}}}'

### Index the Pointer Field

In [85]:
pointer = Pointer(
    Structure(
        size=Decimal(16),
        item=Pointer(String(14))))

In [86]:
pointer.initialize_fields({
    'value': 0x1,
    'data': {
        'size': 14,
        'item': {'value': 0x10,
                 'data': 'Konfoo is Fun'}}})

In [87]:
pointer.to_list('index', nested=True)

[('Pointer.field',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.size',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.item',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.item.data',
  Index(byte=0, bit=0, address=0, base_address=0, update=False))]

In [88]:
pointer.to_csv('index.byte', 'index.bit', 'index.address', fieldnames=('id', 'index', 'offset', 'address'), nested=True)

[{'id': 'Pointer.field', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Pointer.data.size', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Pointer.data.item', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Pointer.data.item.data', 'index': 0, 'offset': 0, 'address': 0}]

In [89]:
pointer.to_json('index', nested=True)

'{"value": [0, 0, 0, 0, false], "data": {"size": [0, 0, 0, 0, false], "item": {"value": [0, 0, 0, 0, false], "data": [0, 0, 0, 0, false]}}}'

In [90]:
pointer.index_field(index=Index())

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [91]:
pointer.index_field()

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [92]:
pointer.to_list('index', nested=True)

[('Pointer.field',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.size',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.item',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.item.data',
  Index(byte=0, bit=0, address=0, base_address=0, update=False))]

In [93]:
pointer.to_csv('index.byte', 'index.bit', 'index.address', fieldnames=('id', 'index', 'offset', 'address'), nested=True)

[{'id': 'Pointer.field', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Pointer.data.size', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Pointer.data.item', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Pointer.data.item.data', 'index': 0, 'offset': 0, 'address': 0}]

In [94]:
pointer.to_json('index', nested=True)

'{"value": [0, 0, 0, 0, false], "data": {"size": [0, 0, 0, 0, false], "item": {"value": [0, 0, 0, 0, false], "data": [0, 0, 0, 0, false]}}}'

### Index the Data Object

In [95]:
pointer = Pointer(
    Structure(
        size=Decimal(16),
        item=Pointer(String())))

In [96]:
pointer.initialize_fields({
    'value': 0x1,
    'data': {
        'size': 14,
        'item': {'value': 0x10,
                 'data': 'Konfoo is Fun'}}})

In [97]:
pointer.to_list('index', nested=True)

[('Pointer.field',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.size',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.item',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.item.data',
  Index(byte=0, bit=0, address=0, base_address=0, update=False))]

In [98]:
pointer.to_csv('index.byte', 'index.bit', 'index.address', fieldnames=('id', 'index', 'offset', 'address'), nested=True)

[{'id': 'Pointer.field', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Pointer.data.size', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Pointer.data.item', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Pointer.data.item.data', 'index': 0, 'offset': 0, 'address': 0}]

In [99]:
pointer.to_json('index', nested=True)

'{"value": [0, 0, 0, 0, false], "data": {"size": [0, 0, 0, 0, false], "item": {"value": [0, 0, 0, 0, false], "data": [0, 0, 0, 0, false]}}}'

In [100]:
pointer.index_data()

In [101]:
pointer.to_list('index', nested=True)

[('Pointer.field',
  Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Pointer.data.size',
  Index(byte=0, bit=0, address=1, base_address=1, update=False)),
 ('Pointer.data.item',
  Index(byte=2, bit=0, address=3, base_address=1, update=False)),
 ('Pointer.data.item.data',
  Index(byte=0, bit=0, address=16, base_address=16, update=False))]

In [102]:
pointer.to_csv('index.byte', 'index.bit', 'index.address', fieldnames=('id', 'index', 'offset', 'address'), nested=True)

[{'id': 'Pointer.field', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Pointer.data.size', 'index': 0, 'offset': 0, 'address': 1},
 {'id': 'Pointer.data.item', 'index': 2, 'offset': 0, 'address': 3},
 {'id': 'Pointer.data.item.data', 'index': 0, 'offset': 0, 'address': 16}]

In [103]:
pointer.to_json('index', nested=True)

'{"value": [0, 0, 0, 0, false], "data": {"size": [0, 0, 1, 1, false], "item": {"value": [2, 0, 3, 1, false], "data": [0, 0, 16, 16, false]}}}'

## De-Serializing

In [104]:
pointer = Pointer(
    Structure(
        size=Decimal(16),
        item=Pointer(String(14))))

In [105]:
pointer.to_list(nested=True)

[('Pointer.field', '0x0'),
 ('Pointer.data.size', 0),
 ('Pointer.data.item', '0x0'),
 ('Pointer.data.item.data', '')]

In [106]:
pointer.to_csv(nested=True)

[{'id': 'Pointer.field', 'value': '0x0'},
 {'id': 'Pointer.data.size', 'value': 0},
 {'id': 'Pointer.data.item', 'value': '0x0'},
 {'id': 'Pointer.data.item.data', 'value': ''}]

In [107]:
pointer.to_json(nested=True)

'{"value": "0x0", "data": {"size": 0, "item": {"value": "0x0", "data": ""}}}'

In [108]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x0'
data:
  size: 0
  item:
    value: '0x0'
    data: ''


In [109]:
pointer.bytestream

''

In [110]:
pointer.data.item.bytestream

''

In [111]:
pointer.deserialize(bytes.fromhex('010000000e00100000004b6f6e666f6f2069732046756e00f00f00'))

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [112]:
pointer.to_list(nested=True)

[('Pointer.field', '0x1'),
 ('Pointer.data.size', 0),
 ('Pointer.data.item', '0x0'),
 ('Pointer.data.item.data', '')]

In [113]:
pointer.to_csv(nested=True)

[{'id': 'Pointer.field', 'value': '0x1'},
 {'id': 'Pointer.data.size', 'value': 0},
 {'id': 'Pointer.data.item', 'value': '0x0'},
 {'id': 'Pointer.data.item.data', 'value': ''}]

In [114]:
pointer.to_json(nested=True)

'{"value": "0x1", "data": {"size": 0, "item": {"value": "0x0", "data": ""}}}'

In [115]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x1'
data:
  size: 0
  item:
    value: '0x0'
    data: ''


In [116]:
pointer.deserialize(bytes.fromhex('010000000e00100000004b6f6e666f6f2069732046756e00f00f00'), nested=True)

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [117]:
pointer.to_list(nested=True)

[('Pointer.field', '0x1'),
 ('Pointer.data.size', 0),
 ('Pointer.data.item', '0x0'),
 ('Pointer.data.item.data', '')]

In [118]:
pointer.to_csv(nested=True)

[{'id': 'Pointer.field', 'value': '0x1'},
 {'id': 'Pointer.data.size', 'value': 0},
 {'id': 'Pointer.data.item', 'value': '0x0'},
 {'id': 'Pointer.data.item.data', 'value': ''}]

In [119]:
pointer.to_json(nested=True)

'{"value": "0x1", "data": {"size": 0, "item": {"value": "0x0", "data": ""}}}'

In [120]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x1'
data:
  size: 0
  item:
    value: '0x0'
    data: ''


In [121]:
pointer.bytestream = '0e0010000000'

In [122]:
pointer.data.item.bytestream = '4b6f6e666f6f2069732046756e00f00f00'

In [123]:
pointer.deserialize(bytes.fromhex('01000000'))

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [124]:
pointer.to_list(nested=True)

[('Pointer.field', '0x1'),
 ('Pointer.data.size', 0),
 ('Pointer.data.item', '0x0'),
 ('Pointer.data.item.data', '')]

In [125]:
pointer.to_csv(nested=True)

[{'id': 'Pointer.field', 'value': '0x1'},
 {'id': 'Pointer.data.size', 'value': 0},
 {'id': 'Pointer.data.item', 'value': '0x0'},
 {'id': 'Pointer.data.item.data', 'value': ''}]

In [126]:
pointer.to_json(nested=True)

'{"value": "0x1", "data": {"size": 0, "item": {"value": "0x0", "data": ""}}}'

In [127]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x1'
data:
  size: 0
  item:
    value: '0x0'
    data: ''


In [128]:
pointer.deserialize(bytes.fromhex('01000000'), nested=True)

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [129]:
pointer.to_list(nested=True)

[('Pointer.field', '0x1'),
 ('Pointer.data.size', 14),
 ('Pointer.data.item', '0x10'),
 ('Pointer.data.item.data', 'Konfoo is Fun')]

In [130]:
pointer.to_csv(nested=True)

[{'id': 'Pointer.field', 'value': '0x1'},
 {'id': 'Pointer.data.size', 'value': 14},
 {'id': 'Pointer.data.item', 'value': '0x10'},
 {'id': 'Pointer.data.item.data', 'value': 'Konfoo is Fun'}]

In [131]:
pointer.to_json(nested=True)

'{"value": "0x1", "data": {"size": 14, "item": {"value": "0x10", "data": "Konfoo is Fun"}}}'

In [132]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x1'
data:
  size: 14
  item:
    value: '0x10'
    data: Konfoo is Fun


### Deserialize the Data Object

In [133]:
pointer = Pointer(
    Structure(
        size=Decimal(16),
        item=Pointer(String(14))), 1)

In [134]:
pointer.to_list(nested=True)

[('Pointer.field', '0x1'),
 ('Pointer.data.size', 0),
 ('Pointer.data.item', '0x0'),
 ('Pointer.data.item.data', '')]

In [135]:
pointer.to_csv(nested=True)

[{'id': 'Pointer.field', 'value': '0x1'},
 {'id': 'Pointer.data.size', 'value': 0},
 {'id': 'Pointer.data.item', 'value': '0x0'},
 {'id': 'Pointer.data.item.data', 'value': ''}]

In [136]:
pointer.to_json(nested=True)

'{"value": "0x1", "data": {"size": 0, "item": {"value": "0x0", "data": ""}}}'

In [137]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x1'
data:
  size: 0
  item:
    value: '0x0'
    data: ''


In [138]:
pointer.bytestream

''

In [139]:
pointer.data.item.bytestream = '4b6f6e666f6f2069732046756e00'

In [140]:
pointer.deserialize_data(bytes.fromhex('0e0010000000'))

Index(byte=6, bit=0, address=7, base_address=1, update=False)

In [141]:
pointer.bytestream

''

In [142]:
pointer.to_list(nested=True)

[('Pointer.field', '0x1'),
 ('Pointer.data.size', 14),
 ('Pointer.data.item', '0x10'),
 ('Pointer.data.item.data', '')]

In [143]:
pointer.to_csv(nested=True)

[{'id': 'Pointer.field', 'value': '0x1'},
 {'id': 'Pointer.data.size', 'value': 14},
 {'id': 'Pointer.data.item', 'value': '0x10'},
 {'id': 'Pointer.data.item.data', 'value': ''}]

In [144]:
pointer.to_json(nested=True)

'{"value": "0x1", "data": {"size": 14, "item": {"value": "0x10", "data": ""}}}'

In [145]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x1'
data:
  size: 14
  item:
    value: '0x10'
    data: ''


In [146]:
pointer.data.item.deserialize_data()

Index(byte=14, bit=0, address=30, base_address=16, update=False)

In [147]:
pointer.to_list(nested=True)

[('Pointer.field', '0x1'),
 ('Pointer.data.size', 14),
 ('Pointer.data.item', '0x10'),
 ('Pointer.data.item.data', 'Konfoo is Fun')]

In [148]:
pointer.to_csv(nested=True)

[{'id': 'Pointer.field', 'value': '0x1'},
 {'id': 'Pointer.data.size', 'value': 14},
 {'id': 'Pointer.data.item', 'value': '0x10'},
 {'id': 'Pointer.data.item.data', 'value': 'Konfoo is Fun'}]

In [149]:
pointer.to_json(nested=True)

'{"value": "0x1", "data": {"size": 14, "item": {"value": "0x10", "data": "Konfoo is Fun"}}}'

In [150]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x1'
data:
  size: 14
  item:
    value: '0x10'
    data: Konfoo is Fun


## Serializing

In [151]:
pointer.to_list(nested=True)

[('Pointer.field', '0x1'),
 ('Pointer.data.size', 14),
 ('Pointer.data.item', '0x10'),
 ('Pointer.data.item.data', 'Konfoo is Fun')]

In [152]:
pointer.to_csv(nested=True)

[{'id': 'Pointer.field', 'value': '0x1'},
 {'id': 'Pointer.data.size', 'value': 14},
 {'id': 'Pointer.data.item', 'value': '0x10'},
 {'id': 'Pointer.data.item.data', 'value': 'Konfoo is Fun'}]

In [153]:
pointer.to_json(nested=True)

'{"value": "0x1", "data": {"size": 14, "item": {"value": "0x10", "data": "Konfoo is Fun"}}}'

In [154]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x1'
data:
  size: 14
  item:
    value: '0x10'
    data: Konfoo is Fun


In [155]:
pointer.bytestream = ''
pointer.bytestream

''

In [156]:
pointer.data.item.bytestream = ''
pointer.data.item.bytestream

''

In [157]:
bytestream = bytearray()
bytestream.hex()

''

In [158]:
pointer.serialize(bytestream)

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [159]:
pointer.bytestream

''

In [160]:
pointer.data.item.bytestream

''

In [161]:
bytes(pointer).hex()

'01000000'

In [162]:
bytestream.hex()

'01000000'

In [163]:
bytestream = bytearray()
bytestream.hex()

''

In [164]:
pointer.serialize(bytestream, nested=True)

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [165]:
pointer.bytestream

'0e0010000000'

In [166]:
pointer.data.item.bytestream

'4b6f6e666f6f2069732046756e00'

In [167]:
bytes(pointer).hex()

'01000000'

In [168]:
bytestream.hex()

'01000000'

### Serialize the Data Object

In [169]:
pointer.to_list(nested=True)

[('Pointer.field', '0x1'),
 ('Pointer.data.size', 14),
 ('Pointer.data.item', '0x10'),
 ('Pointer.data.item.data', 'Konfoo is Fun')]

In [170]:
pointer.to_csv(nested=True)

[{'id': 'Pointer.field', 'value': '0x1'},
 {'id': 'Pointer.data.size', 'value': 14},
 {'id': 'Pointer.data.item', 'value': '0x10'},
 {'id': 'Pointer.data.item.data', 'value': 'Konfoo is Fun'}]

In [171]:
pointer.to_json(nested=True)

'{"value": "0x1", "data": {"size": 14, "item": {"value": "0x10", "data": "Konfoo is Fun"}}}'

In [172]:
to_yaml(pointer, nested=True, flow_style=False)

value: '0x1'
data:
  size: 14
  item:
    value: '0x10'
    data: Konfoo is Fun


In [173]:
pointer.bytestream = ''
pointer.bytestream

''

In [174]:
pointer.data.item.bytestream = ''
pointer.data.item.bytestream

''

In [175]:
pointer.serialize_data().hex()

'0e0010000000'

In [176]:
pointer.bytestream

''

In [177]:
pointer.data.item.serialize_data().hex()

'4b6f6e666f6f2069732046756e00'

In [178]:
pointer.data.item.bytestream

''

## Pointer Members

### Access the Data Object of a Pointer Field

In [179]:
pointer.data

{'size': Decimal(index=Index(byte=0, bit=0, address=1, base_address=1, update=False), alignment=Alignment(byte_size=2, bit_offset=0), bit_size=16, value=14),
 'item': Pointer(index=Index(byte=2, bit=0, address=3, base_address=1, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=32, value='0x10')}

### Access the Attributes of a Pointer Field

In [180]:
pointer.name

'Pointer32'

In [181]:
pointer.value

'0x1'

In [182]:
pointer.bit_size

32

In [183]:
pointer.alignment

Alignment(byte_size=4, bit_offset=0)

In [184]:
pointer.alignment.byte_size

4

In [185]:
pointer.alignment.bit_offset

0

In [186]:
pointer.byte_order

Byteorder.auto = 'auto'

In [187]:
pointer.byte_order.value

'auto'

In [188]:
pointer.index

Index(byte=0, bit=0, address=0, base_address=0, update=False)

In [189]:
pointer.index.byte

0

In [190]:
pointer.index.bit

0

In [191]:
pointer.index.address

0

In [192]:
pointer.index.base_address

0

In [193]:
pointer.index.update

False

## Pointer Fields

### View Field Attributes

In [194]:
pointer.view_fields()

{'value': '0x1', 'data': {'size': 14, 'item': '0x10'}}

In [195]:
pointer.view_fields(nested=True)

{'value': '0x1',
 'data': {'size': 14, 'item': {'value': '0x10', 'data': 'Konfoo is Fun'}}}

In [196]:
pointer.view_fields('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order.name')

{'name': 'Pointer32',
 'bit_size': 32,
 'value': '0x1',
 'index': Index(byte=0, bit=0, address=0, base_address=0, update=False),
 'alignment': Alignment(byte_size=4, bit_offset=0),
 'byte_order.name': 'auto',
 'data': {'size': {'name': 'Decimal16',
   'bit_size': 16,
   'value': 14,
   'index': Index(byte=0, bit=0, address=1, base_address=1, update=False),
   'alignment': Alignment(byte_size=2, bit_offset=0),
   'byte_order.name': 'auto'},
  'item': {'name': 'Pointer32',
   'bit_size': 32,
   'value': '0x10',
   'index': Index(byte=2, bit=0, address=3, base_address=1, update=False),
   'alignment': Alignment(byte_size=4, bit_offset=0),
   'byte_order.name': 'auto'}}}

### View as a JSON string

In [197]:
pointer.to_json()

'{"value": "0x1", "data": {"size": 14, "item": "0x10"}}'

In [198]:
print(pointer.to_json(indent=2))

{
  "value": "0x1",
  "data": {
    "size": 14,
    "item": "0x10"
  }
}


In [199]:
pointer.to_json(nested=True)

'{"value": "0x1", "data": {"size": 14, "item": {"value": "0x10", "data": "Konfoo is Fun"}}}'

In [200]:
print(pointer.to_json(nested=True, indent=2))

{
  "value": "0x1",
  "data": {
    "size": 14,
    "item": {
      "value": "0x10",
      "data": "Konfoo is Fun"
    }
  }
}


In [201]:
pointer.to_json('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order')

'{"name": "Pointer32", "bit_size": 32, "value": "0x1", "index": [0, 0, 0, 0, false], "alignment": [4, 0], "byte_order": "auto", "data": {"size": {"name": "Decimal16", "bit_size": 16, "value": 14, "index": [0, 0, 1, 1, false], "alignment": [2, 0], "byte_order": "auto"}, "item": {"name": "Pointer32", "bit_size": 32, "value": "0x10", "index": [2, 0, 3, 1, false], "alignment": [4, 0], "byte_order": "auto"}}}'

In [202]:
print(pointer.to_json('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order', indent=2))

{
  "name": "Pointer32",
  "bit_size": 32,
  "value": "0x1",
  "index": [
    0,
    0,
    0,
    0,
    false
  ],
  "alignment": [
    4,
    0
  ],
  "byte_order": "auto",
  "data": {
    "size": {
      "name": "Decimal16",
      "bit_size": 16,
      "value": 14,
      "index": [
        0,
        0,
        1,
        1,
        false
      ],
      "alignment": [
        2,
        0
      ],
      "byte_order": "auto"
    },
    "item": {
      "name": "Pointer32",
      "bit_size": 32,
      "value": "0x10",
      "index": [
        2,
        0,
        3,
        1,
        false
      ],
      "alignment": [
        4,
        0
      ],
      "byte_order": "auto"
    }
  }
}


### Write to a JSON file

In [203]:
pointer.write_json('Pointer.json', nested=True)

### List the Field Items

In [204]:
pointer.field_items()

[('field',
  Pointer(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=32, value='0x1')),
 ('data.size',
  Decimal(index=Index(byte=0, bit=0, address=1, base_address=1, update=False), alignment=Alignment(byte_size=2, bit_offset=0), bit_size=16, value=14)),
 ('data.item',
  Pointer(index=Index(byte=2, bit=0, address=3, base_address=1, update=False), alignment=Alignment(byte_size=4, bit_offset=0), bit_size=32, value='0x10'))]

### List the Field Attributes

In [205]:
pointer.to_list()

[('Pointer.field', '0x1'),
 ('Pointer.data.size', 14),
 ('Pointer.data.item', '0x10')]

In [206]:
pointer.to_list(nested=True)

[('Pointer.field', '0x1'),
 ('Pointer.data.size', 14),
 ('Pointer.data.item', '0x10'),
 ('Pointer.data.item.data', 'Konfoo is Fun')]

In [207]:
pointer.to_list('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order')

[('Pointer.field',
  ('Pointer32',
   32,
   '0x1',
   Index(byte=0, bit=0, address=0, base_address=0, update=False),
   Alignment(byte_size=4, bit_offset=0),
   Byteorder.auto = 'auto')),
 ('Pointer.data.size',
  ('Decimal16',
   16,
   14,
   Index(byte=0, bit=0, address=1, base_address=1, update=False),
   Alignment(byte_size=2, bit_offset=0),
   Byteorder.auto = 'auto')),
 ('Pointer.data.item',
  ('Pointer32',
   32,
   '0x10',
   Index(byte=2, bit=0, address=3, base_address=1, update=False),
   Alignment(byte_size=4, bit_offset=0),
   Byteorder.auto = 'auto'))]

In [208]:
pointer.to_dict()

{'Pointer': {'field': '0x1', 'data.size': 14, 'data.item': '0x10'}}

In [209]:
pointer.to_dict(nested=True)

{'Pointer': {'field': '0x1',
  'data.size': 14,
  'data.item': '0x10',
  'data.item.data': 'Konfoo is Fun'}}

In [210]:
pointer.to_dict('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order')

{'Pointer': {'field': ('Pointer32',
   32,
   '0x1',
   Index(byte=0, bit=0, address=0, base_address=0, update=False),
   Alignment(byte_size=4, bit_offset=0),
   Byteorder.auto = 'auto'),
  'data.size': ('Decimal16',
   16,
   14,
   Index(byte=0, bit=0, address=1, base_address=1, update=False),
   Alignment(byte_size=2, bit_offset=0),
   Byteorder.auto = 'auto'),
  'data.item': ('Pointer32',
   32,
   '0x10',
   Index(byte=2, bit=0, address=3, base_address=1, update=False),
   Alignment(byte_size=4, bit_offset=0),
   Byteorder.auto = 'auto')}}

In [211]:
pointer.to_csv()

[{'id': 'Pointer.field', 'value': '0x1'},
 {'id': 'Pointer.data.size', 'value': 14},
 {'id': 'Pointer.data.item', 'value': '0x10'}]

In [212]:
pointer.to_csv(nested=True)

[{'id': 'Pointer.field', 'value': '0x1'},
 {'id': 'Pointer.data.size', 'value': 14},
 {'id': 'Pointer.data.item', 'value': '0x10'},
 {'id': 'Pointer.data.item.data', 'value': 'Konfoo is Fun'}]

In [213]:
pointer.to_csv('name', 'bit_size', 'value', fieldnames=('id', 'type', 'size', 'value'), nested=True)

[{'id': 'Pointer.field', 'type': 'Pointer32', 'size': 32, 'value': '0x1'},
 {'id': 'Pointer.data.size', 'type': 'Decimal16', 'size': 16, 'value': 14},
 {'id': 'Pointer.data.item', 'type': 'Pointer32', 'size': 32, 'value': '0x10'},
 {'id': 'Pointer.data.item.data',
  'type': 'String14',
  'size': 112,
  'value': 'Konfoo is Fun'}]

### Write the Field Attributes to a `.csv` File

In [214]:
pointer.write_csv('Pointer.csv', 'name', 'bit_size', 'value', fieldnames=('id', 'type', 'size', 'value'), nested=True)

### Save the Field Attributes to an `.ini` File

In [215]:
pointer.save('Pointer.ini', nested=True)

### Load the Field Value from an `.ini` File

In [216]:
pointer.load('Pointer.ini', nested=True)

[Pointer]
Pointer.field = 0x1
Pointer.data.size = 14
Pointer.data.item = 0x10
Pointer.data.item.data = Konfoo is Fun
